In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/secondary_data.csv', sep=';')

In [ ]:
df_1 = df.loc[df["class"] == "e"]
df_2 = df.loc[df["class"] == "p"]

In [ ]:
X_1 = df_1.drop("class", axis=1)
X_2 = df_2.drop("class", axis=1)
y_1 = df_1["class"]
y_2 = df_2["class"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.15, random_state=87)
X_train_1, X_validation_1, y_train_1, y_validation_1 = train_test_split(X_train_1, y_train_1, test_size=0.18, random_state=87)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.15, random_state=87)
X_train_2, X_validation_2, y_train_2, y_validation_2 = train_test_split(X_train_2, y_train_2, test_size=0.18, random_state=87)

In [ ]:
X_train_inicial = pd.concat([X_train_1, X_train_2], ignore_index=True)
X_validation_inicial = pd.concat([X_validation_1, X_validation_2], ignore_index=True)
X_test_inicial = pd.concat([X_test_1, X_test_2], ignore_index=True)

y_train = pd.concat([y_train_1, y_train_2], ignore_index=True)
y_validation= pd.concat([y_validation_1, y_validation_2], ignore_index=True)
y_test = pd.concat([y_test_1, y_test_2], ignore_index=True)

#Encodings:

In [ ]:
#Para el target 1 si es e, 0 si es p

In [ ]:
y_train = y_train.map(lambda x: 1 if x == 'e' else 0)
y_validation = y_validation.map(lambda x: 1 if x == 'e' else 0)
y_test = y_test.map(lambda x: 1 if x == 'e' else 0)

In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import category_encoders as ce

In [ ]:
ohe_features = ce.OneHotEncoder(cols = ["does-bruise-or-bleed", "has-ring", "veil-type"], return_df=True)
X_train = ohe_features.fit_transform(X_train_inicial)
X_test = ohe_features.transform(X_test_inicial)
X_validation = ohe_features.transform(X_validation_inicial)

In [ ]:
te_features =  ce.TargetEncoder(cols = ["habitat", "season"], smoothing=2, return_df=True)
X_train = te_features.fit_transform(X_train, y_train)
X_test = te_features.transform(X_test, y_test)
X_validation = te_features.transform(X_validation, y_validation)

In [ ]:
be_features = ce.BinaryEncoder(cols = ["cap-shape", "cap-surface", "cap-color", "gill-attachment", "gill-spacing", "gill-color",
              "stem-color", "ring-type", "stem-root", "stem-surface", "veil-color", "spore-print-color"], return_df=True)
X_train = be_features.fit_transform(X_train)
X_test = be_features.transform(X_test)
X_validation = be_features.transform(X_validation)

#Cargo mis modelos de la parte 3 y 4

In [ ]:
import xgboost as xgb

In [ ]:
xgb_clf = xgb.XGBClassifier()

In [ ]:
xgb_clf.load_model('/content/drive/MyDrive/xgb_clf_mushroom.json')

In [ ]:
xgb_clf

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None,
              feature_types=['float', 'int', 'int', 'int', 'int', 'int', 'int',
                             'int', 'int', 'int', 'int', 'int', 'int', 'int',
                             'int', 'int', 'int', 'int', 'int', 'int', 'int',
                             'int', 'int', '...
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=10, ...)

In [ ]:
from joblib import load

In [ ]:
rf_clf = load('/content/drive/MyDrive/rf_clf_mushroom.joblib')

In [ ]:
rf_clf

RandomForestClassifier(n_estimators=40, random_state=10)

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
voting_clf = VotingClassifier(estimators=[ ('rf_clf', rf_clf), ('xgb_clf', xgb_clf)], voting='soft')

In [ ]:
voting_clf

VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(n_estimators=40,
                                                     random_state=10)),
                             ('xgb_clf',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=['float', 'int',
                                                           'int', 'int', 'int',
                                                           'i...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=10, ...))],
                 voting='soft')

In [ ]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(n_estimators=40,
                                                     random_state=10)),
                             ('xgb_clf',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=['float', 'int',
                                                           'int', 'int', 'int',
                                                           'i...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=10, ...))],
                 voting='soft')

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_validation, voting_clf.predict_proba(X_validation)[:, 1]) #El score tanto para validacion como para test da 1

1.0

In [ ]:
roc_auc_score(y_test, voting_clf.predict_proba(X_test)[:, 1])

1.0